In [1]:
import graphlab

In [2]:
sales = graphlab.SFrame('kc_house_data.gl')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1539158014.log


This non-commercial license of GraphLab Create for academic use is assigned to r38411@yahoo.com and will expire on August 31, 2019.


In [4]:
graphlab.canvas.set_target('ipynb')
sales.print_rows(num_rows=3)

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 | 538000.0 |   3.0    |    2.25   |
| 5631500400 | 2015-02-25 00:00:00+00:00 | 180000.0 |   2.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
|    2570.0   |   7242   |   2    |     0      |  0   |     3     |   7   |    2170    |
|    770.0    | 

In [5]:
train_data, test_data = sales.random_split(.8, seed=0)

In [ ]:
def simple_linear_regression(input_feature, output):
    N = len(input_feature)
    numerator = sum(input_feature * output) - (1.0/N) * (sum(input_feature)*sum(output))
    denominator = mean(input_feature*input_feature) - mean(input_feature) * mean(input_feature)
return (intercept, slope)

i_feature = sales['sqft_living']
out = sales['price']
(simple_intercept, simple_slope) = simple_linear_regression(i_feature, out)
print simple_intercept, simple_slope